Max Feinglass
Principles of Urban Informatics
Homework 5
Assignment 1

In [ ]:
import sys
import geopandas as gp
import pandas as pd
import fiona
import os
import json
import pylab as pl
import statsmodels.formula.api as smf
#s = json.load( open(os.getenv('PUI2015')+'/fbb_matplotlibrc.json') )
#pl.rcParams.update(s)
%pylab inline



Import Local Law 84 Building Benchmarking data for City of New York from: https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/rgfe-8y2z

In [ ]:
nrg = gp.GeoDataFrame.from_csv(os.path.expandvars('$PUI2015/Energy_and_Water_Data_Disclosure_for_Local_Law_84__2013_.csv'))

Import PLUTO shapefile for the City of New York from http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/mn_mappluto_15v1.zip.  Extract the zip file and import MNMapPLUTO.shp.

In [ ]:
bsize = gp.GeoDataFrame.from_file(os.path.expandvars("$PUI2015/MNMapPLUTO.shp"))

Plot a scatter Matrix to examine the data and determine where holes exist.

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix (nrg, s=300, figsize=(16, 16), diagonal='kde')

Create function to identify every time an entry in the LL84 Data canNOT be turned into a floating point.  Create an array, the size of the LL84 data, that flags these instances.  Then covert all the non-convertible entries into "NaN".

In [ ]:
def canconvert(mydata):
    try :
        float(mydata)
        return True
    except ValueError:
        return False    

In [ ]:
#create the indeces array here
vfunc = np.vectorize(canconvert)
indx = vfunc(nrg['Site EUI(kBtu/ft2)'])
indx2 = vfunc(nrg['Reported Property Floor Area (Building(s)) (ft²)'])

In [ ]:
nrg['Site EUI(kBtu/ft2)'][~indx]=float('NaN')
nrg['Reported Property Floor Area (Building(s)) (ft²)'][~indx2]=float('NaN')

Create a column that can act as a common 'key' between the LL84 data and the PLUTO data.  This will allow us to 'join' one data set on the other consistently.  Within this merged data frame, change the 'SITE EUI' and 'Floor' data into floats, and then multiply the EUI value by the floor space value to yield a total energy consumption value.

In [ ]:
nrg['BBL'] = nrg['NYC Borough, Block, and Lot (BBL)']

bblnrgdata = pd.merge(nrg, bsize, on=['BBL'])

bblnrgdata['Site EUI(kBtu/ft2)'] = bblnrgdata['Site EUI(kBtu/ft2)'].astype(float)
bblnrgdata['Reported Property Floor Area (Building(s)) (ft²)'] = bblnrgdata['Reported Property Floor Area (Building(s)) (ft²)'].astype(float)

bblnrgdata['energy_total'] = bblnrgdata['Site EUI(kBtu/ft2)'] * bblnrgdata['Reported Property Floor Area (Building(s)) (ft²)']

Segment the data so that we are only looking at buildings that use more than 1000 kBTU's and have more than 5 units (but less than a thosuand units).  

In [ ]:
bblnrgdata_cut = bblnrgdata[(bblnrgdata.energy_total > 1000) &
                            (bblnrgdata.energy_total < 1e10) & 
                            (bblnrgdata.UnitsTotal > 5) & 
                            (bblnrgdata.UnitsTotal < 1000)]

Now we plot the total amount of energy used by each building against the total number of units in each building.  The bunching of the data makes using log scaling an attractive way of visualizing the data.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Total Units per Building')
ax.set_ylabel('Total Energy Consumption per Building')
ax.set_title('Yearly Energy Consumption by Buildings in New York City')
plt.xlim(1,1500)
plt.ylim(1000,1e10)
plt.scatter(bblnrgdata_cut['UnitsTotal'], bblnrgdata_cut['energy_total']) 

Now regress the independent variable, the units in the building, against a the dependent variable, yearly energy consumption using an ordinary least squares method.  This creates a model that can predict energy consumption by the unit count in a given building. 

In [ ]:
x = np.log10(bblnrgdata_cut['UnitsTotal'])
y = np.log10(bblnrgdata_cut['energy_total'])

data = pd.DataFrame({'x':x, 'y':y})
data.x = data.x.replace([np.inf, -np.inf], np.nan)
data.y = data.y.replace([np.inf, -np.inf], np.nan)

model = smf.ols(formula = 'y ~ x', data = data, missing = 'drop').fit()
print model.summary()

The result is a model with low P-Values and reasonable T-Scores.  The R^2 statistic lets us know unit size explains about 12% of energy use.  Now we would like to plot that line of best fit on our scatter plot.

In [ ]:
idx = np.isfinite(data.x) & np.isfinite(data.y)
m, b = np.polyfit(data.x[idx], data.y[idx], 1)

fig = plt.figure(figsize=(10, 10))
plt.plot(x, y, 'r', x, m*x+b, 'b', linewidth = '3')
plt.xlabel('Total Number of Units Per Building')
plt.ylabel('Total Energy Consumed in One Year')
plt.title('OLS Model of Building Energy Consumption Based on Unit Count')

Now we switch the dependent and independent variable assignments to see if this creates a better model.  Since we are comparing a space with only 2 variables, we should see no change in how one variable's relationship affects the others.  We would expect all relevant regression statistics to remain the same.

In [ ]:
y = np.log10(bblnrgdata_cut['UnitsTotal'])
x = np.log10(bblnrgdata_cut['energy_total'])

data = pd.DataFrame({'x':x, 'y':y})
data.x = data.x.replace([np.inf, -np.inf], np.nan)
data.y = data.y.replace([np.inf, -np.inf], np.nan)
data.to_csv('regress.csv')

model = smf.ols(formula = 'y ~ x', data = data, missing = 'drop').fit()
print model.summary()

In [ ]:
idx = np.isfinite(data.x) & np.isfinite(data.y)
m, b = np.polyfit(data.x[idx], data.y[idx], 1)

fig = plt.figure(figsize=(10, 10))
plt.plot(x, y, 'r', x, m*x+b, 'b', linewidth = '5')
plt.ylabel('Total Units')
plt.xlabel('Total Energy')
plt.title('Total Units vs Total Energy')

As expected, neither arrangement of dependent and independent variables changes the ordinary least squares model.  The R^2 value remains .139 for each model.  This also implies it is impossible to a likelihood ratio test because the parameters of the models would be identical.  That does not allow sufficient degrees of freedom for a meaningful test.

Now create a "polyfit" line that uses a second linear line to create a "model within a model".  Instead of a single linear line that minimizes the loss function, here two lines are created in different parts of the domain to minimize different portions of the data.  This should produce a better model as it will 'pass through' more of the data, and therefore explain more of the correlation.

In [ ]:
x = np.log10(bblnrgdata_cut['UnitsTotal'])
y = np.log10(bblnrgdata_cut['energy_total'])

data = pd.DataFrame({'x':x, 'y':y})
data.x = data.x.replace([np.inf, -np.inf], np.nan)
data.y = data.y.replace([np.inf, -np.inf], np.nan)

idx = np.isfinite(data.x) & np.isfinite(data.y)
b2, b1, b0 = np.polyfit(data.x[idx], data.y[idx], 2)

In [ ]:
x_dum = np.array([1, 2, 3])
y_star = b2 * x_dum ** 2 + b1 * x_dum + b0

fig = plt.figure(figsize=(10, 10))
pl.plot(x_dum, y_star, 'yo', linewidth = '5')
pl.scatter(x,y)
pl.xlabel('Units Per Building')
pl.ylabel('Energy Consumption by Building')
pl.title('Polyline Model of Building Energy Consumption')

In [ ]:
model = smf.ols(formula='y ~ x + I(x**2)', data = data)
fit = model.fit()
fit.summary()

Indeed, as we excepted, this model is a better fit to our data, due to the multiple lines being closer to our data.  The R^2 of 0.238 is better than the single line’s R^2 model of 0.139.  In addition, the adjusted R^2 being similar to the R^2 indicates that the addition of second linear line was a meaningful contribution to explaining the energy use.  The likelihood test is now possible as the two lines present us with an adjustable degree of freedom.  

I wanted to keep this last graph in, just beacuse it looked so cool!

In [ ]:
bblnrgdata_cut['YearBuilt'][bblnrgdata_cut['YearBuilt'] < 1800] = 1800

bblnrgdata_cut.plot(kind='scatter',x='energy_total',
                    y='UnitsTotal', fontsize=22, colormap='gnuplot2', 
                    alpha = 1, marker='o',  figsize=(16, 14), loglog=True,  
                    xlim=(1000,1e11), ylim=(1,1000), c=bblnrgdata_cut['YearBuilt']-1900, 
                    s=bblnrgdata_cut['UnitsTotal']/bblnrgdata_cut['UnitsRes']*100)